In [8]:
import cv2
from sklearn.ensemble import AdaBoostClassifier
from sklearn.datasets import make_classification
from sklearn.tree import DecisionTreeClassifier

In [14]:
X, y = make_classification(n_samples=1000, n_features=4,
                            n_informative=2, n_redundant=0,
                            random_state=0, shuffle=False)
base_classifier=DecisionTreeClassifier(max_depth=2)
clf = AdaBoostClassifier(base_classifier, 2048, random_state=0)
print('X',y.shape)
clf.fit(X, y)
clf.predict([[0, 0, 0, 0]])
#clf.score(X, y)

X (1000,)


1